In [1]:
from matplotlib import pyplot as plt
import numpy as np
from Apollo_components import ReadInputsfromFile, ProcessInputs, MakeObservation, MakePlanet
# from create_opacity_catalog import create_opacity_catalog

from test_cython import calculate_Rayleigh_scattering

# from chemlib import Compound

Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


In [2]:
inputs = ReadInputsfromFile("/Users/arthur/Documents/Astronomy/2019/Retrieval/Code/Results/2M2236/spectral_range/2M2236.Piette.G395H.retrieved.ensemble.2024-01-22.dat")

Reading in parameters.


In [3]:
processed_inputs = ProcessInputs(**inputs)

Reading in observations.


FileNotFoundError: [Errno 2] No such file or directory: '/Volumes/Research/Opacities_0v10/gases/h2o.jwst50k.dat'

In [ ]:
processed_inputs.keys()

In [ ]:
dir(processed_inputs["MakePlanet_kwargs"])

In [ ]:
planet = MakePlanet(processed_inputs["MakePlanet_kwargs"])

In [ ]:
get_model = planet.MakeModel(processed_inputs["MakeModel_initialization_kwargs"])

In [ ]:
planet.cross_sections

In [ ]:
processed_inputs["parameters"]

In [ ]:
model = get_model(processed_inputs["parameters"])

In [ ]:
planet.cross_sections

In [ ]:
scattering = calculate_Rayleigh_scattering(planet.cross_sections, np.linspace(1.1, 2.5, num=20000), 71)

In [ ]:
scattering

In [ ]:
observation = MakeObservation(processed_inputs["ModelObservable_initialization_kwargs"])

In [ ]:
obs_args = [np.asarray(model[0]), processed_inputs["parameters"][0], "Rad", 0]

In [ ]:
observed_spectrum = observation(*obs_args)

In [ ]:
plt.plot(observed_spectrum[0])

In [ ]:
no_nitrogen = processed_inputs["parameters"].copy()

In [ ]:
no_nitrogen[slice(5, 7)] = -12

In [ ]:
no_nitrogen

In [ ]:
no_N_model = get_model(no_nitrogen)

In [ ]:
observation = MakeObservation(processed_inputs["ModelObservable_initialization_kwargs"])

In [ ]:
no_N2_obs_args = [np.asarray(no_N_model[0]), processed_inputs["parameters"][0], "Rad", 0]

In [ ]:
no_N2_observed_spectrum = observation(*no_N2_obs_args)

In [ ]:
plt.plot(observed_spectrum[0])
plt.plot(no_N2_observed_spectrum[0])

In [ ]:
wave_lo, wave_hi, _, _, _, _ = np.genfromtxt("data/rory_APOLLO_template.dat").T

In [ ]:
wave_mid = (wave_lo+wave_hi)/2

In [ ]:
plt.plot(wave_mid, observed_spectrum[0])
plt.plot(wave_mid, no_N2_observed_spectrum[0])

In [ ]:
for_rory = np.c_[wave_mid, observed_spectrum[0], no_N2_observed_spectrum[0]]

In [ ]:
np.savetxt("540K_solarCtoO_FeonH0.25_solarN_vs_noN.npy", for_rory, "%.6e")

In [ ]:
x = np.loadtxt("540K_solarCtoO_FeonH0.25_solarN_vs_noN.npy")

In [ ]:
x

In [ ]:
opacity_catalogs = create_opacity_catalog("/Volumes/ResearchStorage/Opacities_0v10/gases/")

In [ ]:
opacity_catalogs["nir"].load_tables(["h2o", "ch4", "nh3"])

In [ ]:
opacity_catalogs["nir"]

In [ ]:
c_tables = opacity_catalogs["nir"].tables_as_c_format()

In [ ]:
#compounds = []
#for specie in opacity_catalogs["nir"].species:
#    try:
#        compound = Compound(specie.upper())
#    except IndexError:
#        compound = None

#    compounds.append(compound)